In [1]:
import pickle
import pandas as pd
import re
from tqdm import tqdm

In [2]:
import time
import datetime
import seaborn as sns
import numpy as np

In [3]:
def replace_logical(s):
    if s:
        return 1
    else:
        return 0
def transformYear(val):
    #year = datetime.datetime.fromtimestamp(int(df_train.date_created[0])).year
    year = datetime.datetime.fromtimestamp(int(val)).year
    return year

def transformMonth(val):
    month = datetime.datetime.fromtimestamp(val).month
    return month

def get_latitude(val):
    res = val['latitude']
    return res

def get_longitude(val):
    res = val['longitude']
    return res

def preprocess_data(df_input):
    df_output = df_input
    df_output = pd.get_dummies(df_output)
    return df_output

def data_initial_preproc (df_input, sample = 'train'):
    
    df_output = df_input
    df_output['can_buy'] = df_output['can_buy'].map(lambda s: replace_logical(s))
    df_output['can_promote'] = df_output['can_promote'].map(lambda s: replace_logical(s))
    df_output['contacts_visible'] = df_output['contacts_visible'].map(lambda s: replace_logical(s))
    df_output['mortgage_available'] = df_output['mortgage_available'].map(lambda s: replace_logical(s))
    df_output['delivery_available'] = df_output['delivery_available'].map(lambda s: replace_logical(s))
    df_output['payment_available'] = df_output['payment_available'].map(lambda s: replace_logical(s))
    df_output['year']=df_output['date_created'].map(transformYear)
    df_output['month']=df_output['date_created'].map(transformMonth)
    
    df_output = df_output.drop(['date_created', 'description', 'fields','id', 'images', 'subway'], axis = 1)
    df_output = df_output.drop(['can_buy', 'can_promote', 'category'], axis = 1)
    df_output = df_output.drop(['name'], axis = 1)
    df_output['latitude'] = df_output['location'].map(get_latitude)
    df_output['longitude'] = df_output['location'].map(get_longitude)
    df_output = df_output.drop(['location'],axis = 1)
    if sample == 'train': 
        ### выбросим цены за перелами 99% персентиля
        tmp = df_train[['price', 'subcategory']].groupby('subcategory').quantile(q=[0.01, 0.99]).reset_index()
        list_df = []
        for i in tqdm(list(set(df_train['subcategory']))):
            list_df.append(
            df_train[
                (df_train['subcategory']==i)&
                (df_train['price'] >=list(tmp[(tmp['subcategory'] == i) & (tmp['level_1'] == 0.01)].price)[0])&
                (df_train['price'] <=list(tmp[(tmp['subcategory'] == i) & (tmp['level_1'] == 0.99)].price)[0])])
        df_without_bad = pd.concat(list_df)
        
        df_output['price'] = df_output['price']/100
        df_output = df_output[df_output['price']>0]
        df_output['price'] = np.log10(df_output['price'] + 1)
    
    df_output['subcategory'] = df_output['subcategory'].map(lambda s: 'S' + str(s))
    df_output = df_output.pipe(preprocess_data)
    
    return df_output

In [4]:
df_train = pickle.load(open("./train_sample.pckl",'rb'))

In [5]:
df_train.shape

(100000, 17)

In [6]:
df_train.shape

(100000, 17)

In [7]:
df_train = df_train.pipe(data_initial_preproc)

100%|███████████████████████████████████████████████████████████████████████████████| 248/248 [00:01<00:00, 161.02it/s]


In [ ]:
#price_ranges = pd.DataFrame()
#for i, k in enumerate(df_train['subcategory'].unique()):
#    print (i, k, df_train[df_train['subcategory']==k]['price'].median(), df_train[df_train['subcategory']==k]['price'].max(), df_train[df_train['subcategory']==k]['price'].std())
#    price_ranges[k] = df_train[df_train['subcategory']==k]['price'].median() +1.5*df_train[df_train['subcategory']==k]['price'].std()
    

In [60]:
#for i in df_train['subcategory'].unique():

In [8]:
df_train.head(1)

,contacts_visible,delivery_available,mortgage_available,payment_available,price,year,month,latitude,longitude,subcategory_S10001,...,subcategory_S907,subcategory_S908,subcategory_S909,subcategory_S910,subcategory_S911,subcategory_S912,subcategory_S914,subcategory_S919,subcategory_S920,subcategory_S921
0,1,0,0,1,3.30103,2017,4,55.806888,37.546077,0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
df_train.shape

(97193, 257)

In [10]:
X = df_train.drop(['price'],axis=1).values
y = df_train['price'].values

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)

In [13]:
from sklearn.metrics import mean_squared_error

In [14]:
#from sklearn.linear_model import LinearRegression
#model = LinearRegression(n_jobs = 1000, normalize = True)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#result = mean_squared_error(y_test, y_pred)
#result

In [15]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=50, n_jobs=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [16]:
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=5, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [17]:
result = mean_squared_error(y_test, y_pred)

In [18]:
result

0.23338156672673294

In [19]:
pickle.dump(model, open("model.pckl", "wb"))

In [20]:
df_test = pickle.load(open("./test_hack.pckl",'rb'))

In [21]:
df_test_id = df_test['id']

In [22]:
df_test = data_initial_preproc(df_test, sample='test')

In [23]:
X_pred = df_test.values

In [24]:
X_pred

array([[ 1.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [25]:
y_hat = model.predict(X_pred)

In [26]:
y_hat

array([ 3.33223841,  3.65815782,  3.5270032 , ...,  2.89583916,
        3.23034939,  3.10192783])

In [27]:
y_hat.shape

(749525,)

In [28]:
result = 10**y_hat - 1

In [29]:
submission = pd.DataFrame()
submission.loc[:,'id'] = df_test_id.reset_index()['id']
submission.loc[:,'price'] = result
submission.to_csv('submission.csv', sep=',', index=False, encoding='utf-8')